## Лабораторная работа 2

In [1]:
import pandas as pd
import numpy as np
import itertools
import math
from IPython.display import display
from cmath import isnan, nan
from numpy import dot
from numpy.linalg import norm

from turtle import color
from unicodedata import name
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.express as px

#### Получить исходные данные

In [2]:
pd.set_option('display.max_columns', None)
df0 = pd.read_csv('../recSystem_dataset/perfume/data.csv', delimiter=',', encoding="utf8")
df = df0.copy(deep=True)

#### Получить характерные для семейств запахи

In [3]:
smellLike = pd.read_csv('../recSystem_dataset/perfume/smells.csv', delimiter=',', encoding="utf8")

smellLike["семейство"] = smellLike["семейство"].map(lambda elem: str(elem).lower())
smellLike["ассоциации"] = smellLike["ассоциации"].map(lambda elem: str(elem).lower().split(';')) + \
                          smellLike["характеристики"].map(lambda elem: str(elem).lower().split(';'))

itemAss = list(set(itertools.chain.from_iterable(smellLike["ассоциации"].values)))
itemAss.remove('nan')
itemAss.sort()

for item in itemAss:
    smellLike[item] = smellLike["ассоциации"].map(lambda elem: 1 if item in elem else 0)
del smellLike["ассоциации"]
del smellLike["характеристики"]
smellLike


,семейство,активный,амбра,апельсин,белый чай,бергамот,благовоние,бодрящий,ваниль,виноград,гвоздика,герань,горный воздух,грейпфрут,древесный,дым,жизнерадостный,зеленое яблоко,зеленый чай,земля,игривый,имбирь,интенсивный,карамель,кедр,корица,кофе,куркума,лаванда,ладан,лайм,легкий,лимон,листья,массивный,металл,морозная свежесть,морской,мох,мускус,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роза,роскошный,сандал,свежий,сладкий,сливочный,смола,согревающий,спокойный,спортивный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,восточные,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,древесные,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,цитрусовые,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
3,фруктовые,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,свежие,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,шипровые,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,пряные,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,цветочные,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,фужерные,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
9,мускусные,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


#### Преобразовать параметры (пол, шлейф, концентрация, сегмент)

In [4]:
sexDict = { "м": 0, "у": 0.5, "ж": 1 }
df["пол"] = df["пол"].map(lambda elem: sexDict[elem])

In [5]:
tailDict = { "крайне незаметный": 0, "незаметный": 0.25, "средний": 0.5, "заметный": 0.75, "очень заметный": 1}
df["шлейф"] = df["шлейф"].map(lambda elem: tailDict[elem])

In [6]:
conDict = {"освежающая вода": 0, "одеколон": 0.2, "туалетная вода": 0.4, "парфюмерная вода": 0.6, "духи": 0.8, "масляные духи": 1}
df["концентрация"] = df["концентрация"].map(lambda elem: conDict[elem])

In [7]:
segmentDict = {"масс-маркет": 0, "люкс": 0.5, "нишевая парфюмерия": 1}
df["сегмент"] = df["сегмент"].map(lambda elem: segmentDict[elem])

#### Задать матрицу смежности для парфюмерных школ

In [8]:
countryDict = {"Франция": 0, "Италия": 1, "ОАЭ": 2, "США": 3, "Россия": 4, "Германия": 5}

countryMatr = np.zeros((len(countryDict), len(countryDict)))
countryMatr[countryDict["Франция"]][countryDict["Италия"]] = countryMatr[countryDict["Италия"]][countryDict["Франция"]] = 0.5
countryMatr[countryDict["Франция"]][countryDict["ОАЭ"]] = countryMatr[countryDict["ОАЭ"]][countryDict["Франция"]] = 0.9
countryMatr[countryDict["Франция"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["Франция"]] = 0.9
countryMatr[countryDict["Франция"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["Франция"]] = 0.8
countryMatr[countryDict["Франция"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Франция"]] = 1

countryMatr[countryDict["Италия"]][countryDict["ОАЭ"]] = countryMatr[countryDict["ОАЭ"]][countryDict["Италия"]] = 0.8
countryMatr[countryDict["Италия"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["Италия"]] = 0.5
countryMatr[countryDict["Италия"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["Италия"]] = 0.9
countryMatr[countryDict["Италия"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Италия"]] = 1

countryMatr[countryDict["ОАЭ"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["ОАЭ"]] = 0.9
countryMatr[countryDict["ОАЭ"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["ОАЭ"]] = 0.8
countryMatr[countryDict["ОАЭ"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["ОАЭ"]] = 1

countryMatr[countryDict["США"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["США"]] = 0.7
countryMatr[countryDict["США"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["США"]] = 1

countryMatr[countryDict["Россия"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Россия"]] = 1

#### Преобразование оставшихся данных

In [9]:
df["верхние ноты"] = df["верхние ноты"].map(lambda elem: str(elem).lower().split(';'))
df["ноты сердца"] = df["ноты сердца"].map(lambda elem: str(elem).lower().split(';'))
df["базовые ноты"] = df["базовые ноты"].map(lambda elem: str(elem).lower().split(';'))
df["запахи"] = df["запахи"].map(lambda elem: str(elem).lower().split(';'))
df["семейства"] = df["семейства"].map(lambda elem: str(elem).lower().split(';'))
df["время года"] = df["время года"].map(lambda elem: str(elem).lower().split(';'))
df["характеристики аромата"] = df["характеристики аромата"].map(lambda elem: str(elem).lower().split(';'))

In [10]:
df["возраст (до 18)"] = df["возраст (до 18)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (до 26)"] = df["возраст (до 26)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (до 35)"] = df["возраст (до 35)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (до 45)"] = df["возраст (до 45)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (после 45)"] = df["возраст (после 45)"].map(lambda elem: 1 if elem == '+' else 0)

In [11]:
df["стойкость минимум (ч)"] = df["стойкость минимум (ч)"].values / max(df["стойкость минимум (ч)"].values)
df["стойкость максимум (ч)"] = df["стойкость максимум (ч)"].values / max(df["стойкость максимум (ч)"].values)

In [12]:
df["цена за 1 мл"] = df["цена"] / df["мл"]
df["цена за 1 мл"] = (df["цена за 1 мл"].values - min(df["цена за 1 мл"].values)) / (max(df["цена за 1 мл"].values) - min(df["цена за 1 мл"].values))

In [13]:
itemSeason = list(set(itertools.chain.from_iterable(df["время года"].values)))
itemSeason.remove('nan')
for item in itemSeason:
    df[item] = df["время года"].map(lambda elem: 1 if item in elem else 0)

In [14]:
itemSmells = list(set(itertools.chain.from_iterable(df["верхние ноты"].values)) | 
             set(itertools.chain.from_iterable(df["ноты сердца"].values))  |
             set(itertools.chain.from_iterable(df["базовые ноты"].values)) |
             set(itertools.chain.from_iterable(df["запахи"].values)) |
             set(itertools.chain.from_iterable(df["характеристики аромата"].values)))
itemSmells.sort()
itemSmells.remove('nan')
for item in itemSmells:
    df[item] = df["верхние ноты"].map(lambda elem: 1 if item in elem else 0) | \
               df["ноты сердца"].map(lambda elem: 1 if item in elem else 0) | \
               df["базовые ноты"].map(lambda elem: 1 if item in elem else 0) | \
               df["запахи"].map(lambda elem: 1 if item in elem else 0) | \
               df["характеристики аромата"].map(lambda elem: 1 if item in elem else 0)

itemFamily = list(set(itertools.chain.from_iterable(df["семейства"].values)))
for item in itemFamily:
    df[item + " семейства"] = df["семейства"].map(lambda elem: 1 if item in elem else 0)

for smell in smellLike.keys().drop('семейство'):
    df[smell] = df["семейства"].map(lambda elem:
        1 if sum(row[smell] for row in smellLike.iloc if row['семейство'] in elem) else 0)

#### Некоторые параметры

In [15]:
F_NAME = "название"
F_DIST = "расстояние"

nameArr = df["название"]

dfTree = pd.DataFrame(columns=["название", "семейства"], data=df[["название", "семейства"]].values)

In [16]:
del df["верхние ноты"]
del df["ноты сердца"]
del df["базовые ноты"]
del df["запахи"]
del df["семейства"]
del df["время года"]
del df["характеристики аромата"]
del df["мл"]
del df["цена"]

In [17]:
display(df)

,название,пол,бренд,страна,сегмент,концентрация,стойкость минимум (ч),стойкость максимум (ч),шлейф,возраст (до 18),возраст (до 26),возраст (до 35),возраст (до 45),возраст (после 45),цена за 1 мл,осень,лето,зима,весна,альдегиды,амбра,амброксан,ананас,апельсин,апельсиновый цвет,арбуз,белые цветы,белый перец,бензоин,бергамот,береза,бобы,бумага,ваниль,ветивер,вишня,гардения,гвоздика,герань,грейпфрут,груша,древесный аккорд,дубовый мох,дыня,жасмин,женственный,жженый сахар,замша,зеленое яблоко,зеленый аккорд,злаки,иланг-иланг,имбирь,ирис,какао,карамель,кардамон,кашмеран,кедр,киви,клубника,кожа,кожаный аккорд,корень фиалки,кориандр,корица,коричневый сахар,кофе,красное яблоко,лаванда,ладан,ландыш,лен,лилия,лимон,личи,магнолия,малина,мандарин,миндаль,можжевельник,мох,мускатный орех,мускус,нарцисс,нежный,нероли,опопанакс,орхидея,османтус,палисандр,пачули,пион,повседневный,полынь,праздничный,ревень,роза,розмарин,розовый перец,сандал,сахар,свежий,сексуальный,силколид,сирень,сладкая вата,сладкий,слива,специи,спортивный,стойкий,тиаре,тимьян,тмин,тропические фрукты,тубероза,уд,фиалка,франжипани,цвет персика,цибетин,цикламен,цитрусы,чай,черная смородина,черника,черный перец,чистый,шалфей,шафран,элеми,яблоко,янтарь,ванильные семейства,цветочные семейства,амбровые семейства,пряные семейства,шипровые семейства,восточные семейства,ароматические семейства,фруктовые семейства,сладкие семейства,фужерные семейства,мускусные семейства,водные семейства,древесные семейства,цитрусовые семейства,активный,белый чай,благовоние,бодрящий,виноград,горный воздух,древесный,дым,жизнерадостный,зеленый чай,земля,игривый,интенсивный,куркума,лайм,легкий,листья,массивный,металл,морозная свежесть,морской,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роскошный,сливочный,смола,согревающий,спокойный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,Shalimar,1.0,Guerlain,Франция,0.5,0.8,0.166667,0.005,0.50,1,1,1,1,1,0.144143,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,3 L'Imperatrice,1.0,Dolce&Gabbana,Италия,0.5,0.4,1.000000,1.000,1.00,1,1,1,1,1,0.079759,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1001 Nights,1.0,Ajmal,ОАЭ,0.5,1.0,0.416667,0.012,0.50,1,1,1,1,1,0.008665,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
3,Sabina,1.0,Arabesque Perfumes,ОАЭ,1.0,0.8,0.416667,0.012,0.50,0,0,1,1,1,0.329302,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Resolute Gold,0.0,Khalis,ОАЭ,0.0,0.6,NaN,NaN,0.50,0,0,0,1,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

In [18]:
del df["название"]
display(df)

,пол,бренд,страна,сегмент,концентрация,стойкость минимум (ч),стойкость максимум (ч),шлейф,возраст (до 18),возраст (до 26),возраст (до 35),возраст (до 45),возраст (после 45),цена за 1 мл,осень,лето,зима,весна,альдегиды,амбра,амброксан,ананас,апельсин,апельсиновый цвет,арбуз,белые цветы,белый перец,бензоин,бергамот,береза,бобы,бумага,ваниль,ветивер,вишня,гардения,гвоздика,герань,грейпфрут,груша,древесный аккорд,дубовый мох,дыня,жасмин,женственный,жженый сахар,замша,зеленое яблоко,зеленый аккорд,злаки,иланг-иланг,имбирь,ирис,какао,карамель,кардамон,кашмеран,кедр,киви,клубника,кожа,кожаный аккорд,корень фиалки,кориандр,корица,коричневый сахар,кофе,красное яблоко,лаванда,ладан,ландыш,лен,лилия,лимон,личи,магнолия,малина,мандарин,миндаль,можжевельник,мох,мускатный орех,мускус,нарцисс,нежный,нероли,опопанакс,орхидея,османтус,палисандр,пачули,пион,повседневный,полынь,праздничный,ревень,роза,розмарин,розовый перец,сандал,сахар,свежий,сексуальный,силколид,сирень,сладкая вата,сладкий,слива,специи,спортивный,стойкий,тиаре,тимьян,тмин,тропические фрукты,тубероза,уд,фиалка,франжипани,цвет персика,цибетин,цикламен,цитрусы,чай,черная смородина,черника,черный перец,чистый,шалфей,шафран,элеми,яблоко,янтарь,ванильные семейства,цветочные семейства,амбровые семейства,пряные семейства,шипровые семейства,восточные семейства,ароматические семейства,фруктовые семейства,сладкие семейства,фужерные семейства,мускусные семейства,водные семейства,древесные семейства,цитрусовые семейства,активный,белый чай,благовоние,бодрящий,виноград,горный воздух,древесный,дым,жизнерадостный,зеленый чай,земля,игривый,интенсивный,куркума,лайм,легкий,листья,массивный,металл,морозная свежесть,морской,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роскошный,сливочный,смола,согревающий,спокойный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,1.0,Guerlain,Франция,0.5,0.8,0.166667,0.005,0.50,1,1,1,1,1,0.144143,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,1.0,Dolce&Gabbana,Италия,0.5,0.4,1.000000,1.000,1.00,1,1,1,1,1,0.079759,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,Ajmal,ОАЭ,0.5,1.0,0.416667,0.012,0.50,1,1,1,1,1,0.008665,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
3,1.0,Arabesque Perfumes,ОАЭ,1.0,0.8,0.416667,0.012,0.50,0,0,1,1,1,0.329302,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,Khalis,ОАЭ,0.0,0.6,NaN,NaN,0.50,0,0,0,1,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0

#### Матрица смежности для семейств (1 уровень)

In [19]:
layer1 = {"восточные": 0, "древесные": 1, "цветочные": 2, "цитрусовые": 3}

treeLayer1 = np.zeros((len(layer1), len(layer1)))
treeLayer1[layer1["восточные"]][layer1["древесные"]] = treeLayer1[layer1["древесные"]][layer1["восточные"]] = 0.5
treeLayer1[layer1["восточные"]][layer1["цветочные"]] = treeLayer1[layer1["цветочные"]][layer1["восточные"]] = 0.7
treeLayer1[layer1["восточные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["восточные"]] = 0.9
treeLayer1[layer1["древесные"]][layer1["цветочные"]] = treeLayer1[layer1["цветочные"]][layer1["древесные"]] = 0.7
treeLayer1[layer1["древесные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["древесные"]] = 0.9
treeLayer1[layer1["цветочные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["цветочные"]] = 0.7

#### Матрица смежности для семейств (2 уровень)

In [20]:
layer2 = {"цветочные": 0, "древесные": 1, "пряные": 2, "ванильные": 3, "фужерные": 4, "водные": 5, "свежие": 6, "фруктовые": 7, "сладкие": 8, "ароматические": 9}

treeLayer2 = np.zeros((len(layer2), len(layer2)))
treeLayer2[layer2["цветочные"]][layer2["древесные"]] = treeLayer2[layer2["древесные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["пряные"]] = treeLayer2[layer2["пряные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["цветочные"]] = 0.6
treeLayer2[layer2["цветочные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["цветочные"]] = 0.2
treeLayer2[layer2["цветочные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["цветочные"]] = 0.3

treeLayer2[layer2["древесные"]][layer2["пряные"]] = treeLayer2[layer2["пряные"]][layer2["древесные"]] = 0.4
treeLayer2[layer2["древесные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["древесные"]] = 0.5
treeLayer2[layer2["древесные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["древесные"]] = 0.3
treeLayer2[layer2["древесные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["древесные"]] = 0.6
treeLayer2[layer2["древесные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["древесные"]] = 0.5
treeLayer2[layer2["древесные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["древесные"]] = 0.6
treeLayer2[layer2["древесные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["древесные"]] = 0.4
treeLayer2[layer2["древесные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["древесные"]] = 0.4

treeLayer2[layer2["пряные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["пряные"]] = 0.5
treeLayer2[layer2["пряные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["пряные"]] = 0.3
treeLayer2[layer2["пряные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["пряные"]] = 0.3
treeLayer2[layer2["пряные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["пряные"]] = 0.3

treeLayer2[layer2["ванильные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["ванильные"]] = 0.5
treeLayer2[layer2["ванильные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["ванильные"]] = 0.7
treeLayer2[layer2["ванильные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["ванильные"]] = 0.6
treeLayer2[layer2["ванильные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["ванильные"]] = 0.4
treeLayer2[layer2["ванильные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["ванильные"]] = 0.3
treeLayer2[layer2["ванильные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["ванильные"]] = 0.5

treeLayer2[layer2["фужерные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["фужерные"]] = 0.5
treeLayer2[layer2["фужерные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["фужерные"]] = 0.5

treeLayer2[layer2["водные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["водные"]] = 0.3
treeLayer2[layer2["водные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["водные"]] = 0.7
treeLayer2[layer2["водные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["водные"]] = 0.6
treeLayer2[layer2["водные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["водные"]] = 0.6

treeLayer2[layer2["свежие"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["свежие"]] = 0.5
treeLayer2[layer2["свежие"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["свежие"]] = 0.6
treeLayer2[layer2["свежие"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["свежие"]] = 0.3

treeLayer2[layer2["фруктовые"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["фруктовые"]] = 0.3
treeLayer2[layer2["фруктовые"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["фруктовые"]] = 0.3

treeLayer2[layer2["сладкие"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["сладкие"]] = 0.6

#### Матрица смежности для семейств (3 уровень)

In [21]:
layer3 = {"свежие": 0, "сладкие": 1, "мускусные": 2, "шипровые": 3, "амбровые": 4, "фруктовые": 5, "-": 6}

treeLayer3 = np.zeros((len(layer3), len(layer3)))
treeLayer3[layer3["свежие"]][layer3["сладкие"]] = treeLayer3[layer3["сладкие"]][layer3["свежие"]] = 0.6
treeLayer3[layer3["свежие"]][layer3["мускусные"]] = treeLayer3[layer3["мускусные"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["свежие"]] = 0.6
treeLayer3[layer3["свежие"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["свежие"]] = 1

treeLayer3[layer3["сладкие"]][layer3["мускусные"]] = treeLayer3[layer3["мускусные"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["сладкие"]] = 1

treeLayer3[layer3["мускусные"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["мускусные"]] = 0.2
treeLayer3[layer3["мускусные"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["мускусные"]] = 0.6
treeLayer3[layer3["мускусные"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["мускусные"]] = 0.6
treeLayer3[layer3["мускусные"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["мускусные"]] = 1

treeLayer3[layer3["шипровые"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["шипровые"]] = 0.6
treeLayer3[layer3["шипровые"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["шипровые"]] = 0.6
treeLayer3[layer3["шипровые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["шипровые"]] = 1

treeLayer3[layer3["амбровые"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["амбровые"]] = 0.7
treeLayer3[layer3["амбровые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["амбровые"]] = 1

treeLayer3[layer3["фруктовые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["фруктовые"]] = 1

In [22]:
layer = [layer1, layer2, layer3]
tree = [treeLayer1, treeLayer2, treeLayer3]

### Данные

In [23]:
excludeFields = ["стойкость минимум (ч)", "стойкость максимум (ч)", "возраст (до 18)", "возраст (до 26)", "возраст (до 35)", 
                 "возраст (до 45)", "возраст (после 45)", "цена за 1 мл", "пол", "бренд", 
                 "страна", "сегмент", "концентрация"]

def getDataFrameAroma(df):
    dfNew = df.copy()
    for elem in excludeFields:
        del dfNew[elem]
    
    return dfNew

def getDataFrameStat(df):
    dfStatParams = pd.DataFrame(columns=excludeFields, data=df[excludeFields].values)
    del dfStatParams["бренд"]
    del dfStatParams["страна"]

    return dfStatParams

#### Получение расстояний

In [24]:
def _complete(v1, v2):
    for i in range(len(v1) - len(v2)):
        v2.append("-")
    return v2

def complete(v1, v2):
    n1, n2 = len(v1), len(v2)
    if n1 > n2:
        v2 = _complete(v1, v2)
    elif n1 < n2:
        v1 = _complete(v2, v1)
    return v1, v2

# Расстояния
def getDistance(v1, v2, nPow):
    res = 0
    for i in range(len(v1)):
        if isnan(v1[i]) or isnan(v2[i]):
            continue
        res += pow(abs(v1[i] - v2[i]), nPow)
    return pow(res, 1 / nPow)

# Манхэттенское расстояние
def getManhattanDistance(v1, v2):
    return getDistance(v1, v2, 1)

# Евклидово расстояние
def getEuclideanDistance(v1, v2):
    return getDistance(v1, v2, 2)

# Косинусное 
def getCos(v1, v2):
    v1T, v2T = v1.copy(), v2.copy()
    n = len(v1)
    indArr = [i for i, (elem1, elem2) in enumerate(zip(v1T, v2T)) if isnan(elem1) or isnan(elem2)]
    v1T[:] = [elem for i, elem in enumerate(v1T) if i not in indArr]
    v2T[:] = [elem for i, elem in enumerate(v2T) if i not in indArr]
    return 1 - dot(v1T, v2T) / (norm(v1T) * norm(v2T))

# Расстояние по дереву
def getTreeDistance(v1, v2):
    res = 0
    resArr = []
    v1T, v2T = v1, v2
    if len(v1) != len(v2):
        v1T, v2T = complete(v1, v2)
    for i in range(len(v1T)):
        res += tree[i][layer[i][v1T[i]]][layer[i][v2T[i]]]
    
    return res / len(tree)

# Сравнение брендов
def getBrandDistance(v1, v2):
    return 0 if v1[0] == v2[0] else 1

# Сравнение стран
def getCountryDistance(v1, v2):
    return countryMatr[countryDict[v1]][countryDict[v2]]

# Найти все похожие
def getSimilarity(id, matr, nameArr):
    data = matr[id]
    res = pd.DataFrame(zip(data, nameArr), index=np.arange(len(matr)), columns=["расстояние", "название"])
    return res.sort_values("расстояние")

In [25]:
# Мера Жаккара
def _getJacquard(v1, v2):
    a = v1.count(1)
    b = v2.count(1)
    c = 0
    for i in range(len(v1)):
        if v1[i] and v2[i]:
            c += 1
    return 1 - c / (a + b - c)


def getJacquard(df, nameArr):
    matrData = df.values.tolist()
    n = len(matrData)
    matrRes = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            matrRes[i][j] = matrRes[j][i] = _getJacquard(matrData[i], matrData[j])
    return matrRes

In [26]:
def calcDistance(f, df):
    matrData = df.values.tolist()
    n = len(matrData)
    matrRes = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            matrRes[i][j] = matrRes[j][i] = f(matrData[i], matrData[j])
    return matrRes / matrRes.max()   


def calcDistanceCompined(df, dfTree):
    dfTree = dfTree["семейства"]
    dfMan = getDataFrameAroma(df)
    dfJac = dfMan.copy()
    del dfJac["шлейф"]

    dfStatParams = getDataFrameStat(df)
    
    matrTree = calcDistance(getTreeDistance, dfTree)
    # matrCos = calcDistance(getCos, dfMan)
    matrEucl = calcDistance(getEuclideanDistance, dfStatParams)
    # matrMan = calcDistance(getManhattanDistance, dfMan)
    matrBrand = calcDistance(getBrandDistance, df["бренд"])
    matrCountry = calcDistance(getCountryDistance, df["страна"])
    matrJac = getJacquard(dfJac, nameArr)

    xTree = matrTree.max()
    # xCos = matrCos.max()
    xEuci = matrEucl.max()
    # xMan = matrMan.max()
    xJac = matrJac.max()
    xBrand = matrBrand.max()
    xCountry = matrCountry.max()


    kJac, kTree, kEuci, kBrand, kCountry = 2, 0.5, 10, 2, 2

    return (kJac * matrJac + kTree * matrTree + matrEucl + kBrand * matrBrand + kCountry * matrCountry) / (kJac * xJac + kTree * xTree + xEuci + kBrand * xBrand + kCountry * xCountry)

#### Графическое изображение

In [27]:
def draw(matrRes, nameArr, title, color='Inferno'):
    fig = px.imshow(matrRes, x=nameArr, y=nameArr, color_continuous_scale=color, title=title)
    fig.update_layout(width=1000, height=1200)
    fig.update_traces(text=nameArr)
    fig.update_xaxes(side="top")
    fig.show()

### Демонстрация расстояний

#### Манхэттонское

In [28]:
matrRes = calcDistance(getManhattanDistance, getDataFrameAroma(df))
draw(matrRes, nameArr, "Манхэттенское расстояние")
# getDifference(11, matrRes, nameArr)

#### Евклидово расстояние

In [29]:
matrRes = calcDistance(getEuclideanDistance, getDataFrameStat(df))
draw(matrRes, nameArr, "Евклидово расстояние")

#### Расстояние по брендам

In [30]:
matrRes = calcDistance(getBrandDistance, df["бренд"])
draw(matrRes, nameArr, "Расстояние по брендам")

#### Расстояние по парфюмерным школам

In [31]:
matrRes = calcDistance(getCountryDistance, df["страна"])
draw(matrRes, nameArr, "Расстояние по парфюмерным школам")

#### Косинусное сходство

In [32]:
matrRes = calcDistance(getCos, getDataFrameAroma(df))
draw(matrRes, nameArr, "Косинусное подобие")

#### Мера Жаккара

In [33]:
tempDF = getDataFrameAroma(df)
del tempDF["шлейф"]

matrRes = getJacquard(tempDF, nameArr)
draw(matrRes, nameArr, "Мера Жаккара")

#### Расстояние по дереву

In [34]:
matrRes = calcDistance(getTreeDistance, dfTree["семейства"])
draw(matrRes, nameArr, "Расстояние по дереву")


#### Кобинированная мера

In [35]:
draw(calcDistanceCompined(df, dfTree), nameArr, "Комбинированная мера")

## Лабораторная работа 3

#### Требуемые параметры

In [36]:
matrSimilarity = calcDistanceCompined(df, dfTree)

### Задача 1

**Вход**: 1 объект (затравочный). 

**Выход**: список рекомендаций, ранжированный по убыванию близости с затравкой. Примените Вашу обобщающую меру близости.

In [37]:
def printRes(arr):
    print("расстояние \t\t\t название")
    for elem in arr:
        for key, value in elem.items():
            print("{0}\t\t{1}".format(value, key))

In [38]:
def _findSimilar(name):
    ind = df0[F_NAME].tolist().index(name)
    
    listSimilarity = getSimilarity(ind, matrSimilarity, nameArr)
    return listSimilarity


def findSimilar(name):
    listSimilarity = _findSimilar(name)

    return listSimilarity[listSimilarity[F_NAME] != name]
    
res = findSimilar("La Petite Robe Noire Ma Robe Cocktail")
print(res)

    расстояние                                   название
23    0.215912  La Petite Robe Noire Ma Robe Sous le Vent
0     0.321552                                   Shalimar
10    0.326026                              Ange ou Demon
21    0.370971           Flora by Gucci Gorgeous Gardenia
11    0.401309                      Samsara Eau de Parfum
14    0.539675                      Lanvin Eclat D Arpege
17    0.608057                                 Channel №5
16    0.649286              Cartier Declaration D Un Soir
9     0.675531               Eau de Lacoste L.12.12 Blanc
1     0.679922                            3 L'Imperatrice
15    0.748310                           MOSCHINO Toy Boy
20    0.764436                               The Only One
6     0.774728                                      Twist
3     0.791740                                     Sabina
4     0.805434                              Resolute Gold
2     0.812658                                1001 Nights
19    0.832928

### Задача 2

**Вход**: массив объектов (лайков). 

**Выход**: сформированный ранжированный список рекомендаций.

In [39]:
from collections import defaultdict


def _findSimilarMany(nameArr):
    recList = []
    for name in nameArr:
        rec = _findSimilar(name)
        recList.append(rec.loc[rec[F_NAME].isin(nameArr) == False])

    dfRes = defaultdict(lambda: 1e2)
    for rec in recList:
        for i, row in rec.iterrows():
            curName = row[F_NAME]
            curDist = row[F_DIST]
            dfRes[curName] = min(dfRes[curName], curDist)

    return dfRes


def findSimilarMany(nameArr):
    resDict = _findSimilarMany(nameArr)
    return sorted([{key: elem} for key, elem in resDict.items()], key=lambda elem: list(elem.values())[0])

res = findSimilarMany(["La Petite Robe Noire Ma Robe Cocktail", "1001 Nights"])
printRes(res)

расстояние 			 название
0.21591159253730813		La Petite Robe Noire Ma Robe Sous le Vent
0.25167162464687304		Oath for Him
0.3215522392103752		Shalimar
0.32602611023416256		Ange ou Demon
0.3357268353923953		Sabina
0.370971124085994		Flora by Gucci Gorgeous Gardenia
0.4013085267251231		Samsara Eau de Parfum
0.5396751018337889		Lanvin Eclat D Arpege
0.6080569739699607		Channel №5
0.6094803133644487		Twist
0.6250350297843664		Resolute Gold
0.649286407130116		Cartier Declaration D Un Soir
0.67553135516767		Eau de Lacoste L.12.12 Blanc
0.6799218074850271		3 L'Imperatrice
0.7483100964045806		MOSCHINO Toy Boy
0.7644358952790559		The Only One
0.8329282057990848		Escentric Molecules Escentric 02
0.8383136823776062		Bad Boy
0.8409492107588658		Renata
0.8464257375303849		Prada Day For Night
0.8736715110228855		CK Everyone
0.9049298305080599		New Your Amber


### Задача 3

**Вход**: Вход: массив затравочных объектов и массив дизлайков.

**Выход**: сформированный ранжированный список рекомендаций.

In [40]:
def delOpposite(dict, nameArr):
    for name in nameArr:
        if name in dict.keys():
            del dict[name]
    
    return dict


def findSimilar(likesArr, dislikesArr):
    likesRec = delOpposite(_findSimilarMany(likesArr), dislikesArr)
    dislikesRec = delOpposite(_findSimilarMany(dislikesArr), likesArr)

    dictRes = {}
    if len(likesArr) == 0:
        for key, elem in dislikesRec.items():
            if elem > 0.7:
                dictRes[key] = elem
        return sorted([{key: elem} for key, elem in dictRes.items()], key=lambda elem: list(elem.values())[0])

    for key in likesRec.keys():
        if likesRec[key] <= dislikesRec[key]:
            dictRes[key] = likesRec[key]
    return sorted([{key: elem} for key, elem in dictRes.items()], key=lambda elem: list(elem.values())[0])


res = findSimilar([],
                  ["Shalimar"])
printRes(res)


расстояние 			 название
0.7238914038274854		Escentric Molecules Escentric 02
0.7360334007375909		Renata
0.7640609606925371		3 L'Imperatrice
0.7827648998837538		New Your Amber
0.8137497287135411		Bad Boy
0.8252506839075587		1001 Nights
0.8712792588735438		Twist
0.8918931827822784		CK Everyone
0.896583497226409		Resolute Gold
0.9025154832293041		Sabina
0.9149402260244764		Oath for Him


### Интерфейс

In [63]:
import ipywidgets as widgets
from tkinter import font
from turtle import onclick, width

import panel as pn

pn.extension()

In [42]:
def getArrFromSeries(data):
    arr = []
    for elem in data:
        arr.append(elem)
    return arr

def getDataFrameFromArr(data, reverse=0):
    resArr = []
    for elem in data:
        for key in elem.keys():
            resArr.append({"Название": key, "Величина схожести": 1 - elem[key] + reverse * (2 * elem[key] - 1)})
            
    return pd.DataFrame(resArr, index=range(1, len(resArr) + 1), columns=["Название", "Величина схожести"])

In [43]:
namesUI = getArrFromSeries(nameArr)

choiceLiked = pn.widgets.MultiChoice(
    name='👍👍👍 Нравится 👍👍👍', 
    value=[],
    width=320,
    options=namesUI)

choiceDisliked = pn.widgets.MultiChoice(
    name='👎👎👎 НЕ нравится 👎👎👎', 
    value=[],
    width=320,
    options=namesUI)

In [44]:
markdownError = pn.pane.Markdown('### Ошибка ввода', style={'font-family': "serif", "text-align": "center", "color": "red"}, width=800, visible=False)
markdownDefault = pn.pane.Markdown("#### Выберете то, что: ", width=800, visible=True)
markdownResultMustTitle = pn.pane.Markdown("#### Попробуйте следующие ароматы: ", width=300, visible=False)
markdownResultMaybeTitle = pn.pane.Markdown("#### Возможно Вам понравятся: ", width=300, visible=False)

bokeh_formatters = {
    "Величина схожести": {'type': 'progress', 'max': 1}
}

tableRecMust = pn.widgets.Tabulator(visible=False, formatters=bokeh_formatters)
tableRecMaybe = pn.widgets.Tabulator(visible=False, formatters=bokeh_formatters)

In [45]:
def _initMustTable(recArr):
    tableRecMust.value = getDataFrameFromArr(recArr)
    
    markdownResultMustTitle.visible = True
    tableRecMust.visible = True


def _initMaybeTable(recArr):
    tableRecMaybe.value = getDataFrameFromArr(recArr)
    
    markdownResultMaybeTitle.visible = True
    tableRecMaybe.visible = True


def _changeStatusError(isError):
    if isError:
        markdownError.visible = True
        markdownDefault.visible = False
        markdownResultMustTitle.visible = False
        markdownResultMaybeTitle.visible = False

        tableRecMust.visible = False
        tableRecMaybe.visible = False
    else:
        markdownError.visible = False
        markdownDefault.visible = True

In [46]:
def _splitMustMaybe(recArr):
    recMust, recMaybe = [], []
    for rec in recArr:
        for key in rec.keys():
            if rec[key] <= 0.5:
                recMust.append(rec)
            else:
                recMaybe.append(rec)
    return recMust, recMaybe


def _splitMustMaybeDict(recDict):
    recMust, recMaybe = [], []
    for name, value in recDict.items():
        if value >= 0.5:
            recMust.append({name: value})
        else:
            recMaybe.append({name: value})
    return recMust, recMaybe


def _isRightInput(arr1, arr2):
    inner = list(set(arr1) & set(arr2))
    return len(inner) == 0


def _getDefaultResult(nameArr):
    resArr = []
    for name in nameArr:
        resArr.append({name: 1})
    return resArr


def _getDefaultResultParams(nameArr):
    resArr = {}
    for name in nameArr:
        resArr[name] = 0
    return resArr

In [47]:
def _getRecommendationArr(likesArr, dislikesArr):
    recArr = None

    if len(likesArr) and len(dislikesArr):
        recArr = findSimilar(likesArr, dislikesArr)
    elif len(likesArr) and len(dislikesArr) == 0:
        recArr = findSimilarMany(likesArr)
    elif len(likesArr) == 0 and len(dislikesArr):
        recArr = findSimilar(likesArr, dislikesArr)
    else:
        recArr = _getDefaultResult(namesUI)
    return recArr

def _giveRecommendation(likesArr, dislikesArr):
    recArr = _getRecommendationArr(likesArr, dislikesArr)

    recMust, recMaybe = _splitMustMaybe(recArr)

    _initMustTable(recMust)
    _initMaybeTable(recMaybe)


def run(a):
    likesArr = choiceLiked.value
    dislikesArr = choiceDisliked.value

    if not _isRightInput(likesArr, dislikesArr):
        _changeStatusError(isError=True)
        return

    _changeStatusError(isError=False)
    _giveRecommendation(likesArr, dislikesArr)


In [48]:
button = pn.widgets.Button(
    name='Готово',
    button_type='success',
    width=50,
    height=40,
    margin=(24, 100, 10, 10))
button.on_click(run)

pLikes = pn.Column(
    markdownDefault,
    markdownError,
    pn.Row(
        pn.Column(choiceLiked, height=800),
        pn.Column(choiceDisliked, height=800),
        button,
        pn.Column(markdownResultMustTitle, 
            tableRecMust,
            markdownResultMaybeTitle, 
            tableRecMaybe))
)
pLikes


BokehModel(combine_events=True, render_bundle={'docs_json': {'b0e2e365-f818-4e5d-a4a1-7b91d38e17de': {'defs': …

### Лабораторная работа 3 и 4

In [49]:
sexWidget = pn.widgets.CheckBoxGroup(
    name='Пол', 
    options=['мужской', 'женский', 'унисекс'],
    inline=False
)
sexElem = pn.Card(
    sexWidget, 
    title='Пол',
    background='WhiteSmoke',
    width=400,
    margin=(10, 30, 10, 10))

In [50]:
countryArr = list(set(df0['страна'].tolist()))

countryWidget = pn.widgets.MultiChoice(
    value=[],
    options=countryArr)
countryElem = pn.Card(
    countryWidget,
    title='Страна-производитель',
    background='WhiteSmoke',
    width=400,
    margin=(10, 30, 10, 10))

In [51]:
brandArr = list(set(df0['бренд'].tolist()))

brandWidget = pn.widgets.MultiChoice(
    value=[],
    options=brandArr)
brandElem = pn.Card(
    brandWidget,
    title='Бренд',
    background='WhiteSmoke',
    width=400,
    margin=(10, 30, 10, 10))

In [52]:
conArr = list(set(df0['концентрация'].tolist()))

conWidget = pn.widgets.MultiChoice(
    value=[],
    options=conArr)
conElem = pn.Card(
    conWidget,
    title='Концентрация',
    background='WhiteSmoke',
    width=400,
    margin=(10, 30, 10, 10))

In [53]:
familyArrT = df0['семейства'].map(lambda elem: elem.split(';')).tolist()
familyArr = []
for elem in familyArrT:
    familyArr += elem
familyArr = list(set(familyArr))

familyWidget = pn.widgets.MultiChoice(
    value=[],
    options=[elem.lower() + " семейства" for elem in familyArr])
familyElem = pn.Card(
    familyWidget,
    title='Семейства',
    background='WhiteSmoke',
    width=400,
    margin=(10, 30, 10, 10))

In [54]:
smellLikeList = smellLike.columns.values.tolist()
smellLikeList.remove("семейство")

smellArr = list(set(smellLikeList) | set(itemSmells))
smellArr.sort()

smellWidget = pn.widgets.MultiSelect(
    value=[],
    size=10,
    options=smellArr)
smellElem = pn.Card(
    smellWidget,
    title='Запахи',
    background='WhiteSmoke',
    width=400,
    margin=(10, 60, 10, 10))


In [55]:
markdownResultMustTitle2 = pn.pane.Markdown("#### Попробуйте следующие ароматы: ", width=300, visible=False)
markdownResultMaybeTitle2 = pn.pane.Markdown("#### Возможно Вам понравятся: ", width=300, visible=False)

tableRecMust2 = pn.widgets.Tabulator(visible=False, formatters=bokeh_formatters)
tableRecMaybe2 = pn.widgets.Tabulator(visible=False, formatters=bokeh_formatters)

In [56]:
def _initMustTable2(recArr):
    tableRecMust2.value = getDataFrameFromArr(recArr, 1)
    
    markdownResultMustTitle2.visible = True
    tableRecMust2.visible = True


def _initMaybeTable2(recArr):
    tableRecMaybe2.value = getDataFrameFromArr(recArr, 1)
    
    markdownResultMaybeTitle2.visible = True
    tableRecMaybe2.visible = True

In [57]:
def _updateSex(sexArr):
    res = []
    for sex in sexArr:
        res.append(sexDict[sex[0]])
    return res

def _updateCon(conArr):
    res = []
    for con in conArr:
        res.append(conDict[con])
    return res

def sortDict(resDict):
    sorted_tuples = sorted(resDict.items(), key=lambda item: item[1], reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}
    return sorted_dict


def _updateResult(dataDict, nameArr, n):
    resDict = {}
    for key, value in dataDict.items():
        if value == 0:
            continue
        resDict[nameArr[key]] = value / n
    
    return sortDict(resDict)
    

def _fromArrDictToDict(arrDict):
    resDict = {}
    for elem in arrDict:
        resDict.update(elem)
    return resDict

In [58]:
def _getRecommendationParams(sexSelected, countrySelected, brandSelected,
                            conSelected, familySelected, smellSelected, df):
    dfColumnsArr = df.columns.values.tolist()
    indexDict = {}
    indexDict[dfColumnsArr.index('пол')] = sexSelected
    indexDict[dfColumnsArr.index('страна')] = countrySelected
    indexDict[dfColumnsArr.index('бренд')] = brandSelected
    indexDict[dfColumnsArr.index('концентрация')] = conSelected
    for family in familySelected:
        indexDict[dfColumnsArr.index(family)] = [1]
    for smell in smellSelected:
        indexDict[dfColumnsArr.index(smell)] = [1]

    matrData = df.values.tolist()
    sDict = {}
    for i in range(len(matrData)):
        s = 0
        for ind in indexDict.keys():
            if matrData[i][ind] in indexDict[ind]:
                s += 1
        sDict[i] = s

    return sDict


def getRecommendationParams(sexSelected, countrySelected, brandSelected,
                            conSelected, familySelected, smellSelected, df):
    nAll = 0
    if len(sexSelected):
        nAll += 1
    if len(countrySelected):
        nAll += 1
    if len(brandSelected):
        nAll += 1
    if len(conSelected):
        nAll += 1
    
    nAll += len(familySelected) + len(smellSelected)
    if nAll == 0:
        recDict = _getDefaultResultParams(namesUI)
    else:
        recDict = _getRecommendationParams(sexSelected, countrySelected, brandSelected, conSelected, familySelected, smellSelected, df)
        recDict = _updateResult(recDict, nameArr, nAll)
    return recDict
    

In [59]:
def _compareLikesParams(likesDict, paramsDict):
    resDict = {}

    for key, value in likesDict.items():
        if key in paramsDict.keys():
            resDict[key] = (value + paramsDict[key]) * 0.5
            del paramsDict[key]
        else:
            resDict[key] = value * 0.5

    if len(paramsDict.keys()):
        for key, value in paramsDict.items():
            resDict[key] = 0.5 * value
    return resDict


def giveRecommendationFull(sexSelected, countrySelected, brandSelected,
                            conSelected, familySelected, smellSelected, likesSelected, dislikesSelected, df):
    recDict = getRecommendationParams(sexSelected, countrySelected, brandSelected, conSelected, familySelected, smellSelected, df)
    if len(likesSelected) or len(dislikesSelected):
        recLikesArr = _getRecommendationArr(likesSelected, dislikesSelected)
        recLikesDict = _fromArrDictToDict(recLikesArr)
        recDict = _compareLikesParams(recLikesDict, recDict)

    recMust, recMaybe = _splitMustMaybeDict(sortDict(recDict))

    _initMustTable2(recMust)
    _initMaybeTable2(recMaybe)


def runFull(a):
    sexSelected = sexWidget.value
    countrySelected = countryWidget.value
    brandSelected = brandWidget.value
    conSelected = conWidget.value
    familySelected = familyWidget.value
    smellSelected = smellWidget.value

    likesSelected = choiceLiked.value
    dislikesSelected = choiceDisliked.value

    if not _isRightInput(likesSelected, dislikesSelected):
        _changeStatusError(isError=True)
        return

    _changeStatusError(isError=False)

    sexSelected = _updateSex(sexSelected)
    conSelected = _updateCon(conSelected)

    giveRecommendationFull(sexSelected, countrySelected, brandSelected, conSelected, familySelected, smellSelected, likesSelected, dislikesSelected, df)

In [60]:
buttonFull = pn.widgets.Button(
    name='Готово',
    button_type='success',
    width=400,
    height=40,
    margin=(24, 100, 10, 10))
buttonFull.on_click(runFull)


elemArr = pn.Column(
    sexElem,
    countryElem,
    brandElem,
    conElem,
    familyElem,
    smellElem,
    buttonFull
)

elemSet = pn.Row (
    elemArr,
    pn.Column(
        markdownResultMustTitle2,
        tableRecMust2,
        markdownResultMaybeTitle2,
        tableRecMaybe2
    )
)

In [64]:
tabs = pn.Tabs(("👍/👎", pLikes), ("⚙️", elemSet))
tabs

BokehModel(combine_events=True, render_bundle={'docs_json': {'45dcbc4b-8dae-453a-b798-c73768e5f23c': {'defs': …